In [1]:
from selenium import webdriver
import chromedriver_binary
from bs4 import BeautifulSoup
import pandas as pd
import time
import geopy

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(options=options)
driver.get("https://www.croma.com/store?location=default")
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [3]:
stores = soup.find('ul', class_ = 'store-list')
phone_numbers = []
for store in stores:
    number = store.find('a')
    if number is not None:
        phone_numbers.append(store.find('a').text.lstrip('Call: '))
    else:
        phone_numbers.append('Not available')

In [4]:
driver.find_element('class name', 'store-card').click()
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
driver.quit()
locator = geopy.Nominatim(user_agent = "myGeocoder", timeout = 10000)

In [5]:
stores = soup.find_all('div', class_ = 'store-data')
df = pd.DataFrame(columns = ['Store Name', 'Address', 'Timings', 'Phone Number', 'Latitude', 'Longitude'])
for store in stores:
    name = store.find('h1', class_ = 'store-name')
    address = store.find('div', class_ = 'address')
    timings = store.find('span', class_ = 'value')
    row = [name, address, timings]
    new_row = []
    for data in row:
        if data is not None:
            new_row.append(data.text)
        else:
            new_row.append("Not available")            
    new_row.append(phone_numbers[len(df.index)])
    
    precision = -4
    tokens = address.text.split(',')
#     time.sleep(1)
    while precision != 0:
        add = ','.join((tokens[precision:]))
        location = locator.geocode(add)
        if location is not None:
            new_row.append(location.latitude)
            new_row.append(location.longitude)
            break
        else:
            precision += 1
    
    if precision == 0:
        new_row.append('Not available')
        new_row.append('Not available')
    df.loc[len(df.index)] = new_row

In [6]:
df

,Store Name,Address,Timings,Phone Number,Latitude,Longitude
0,Memnagar Himalaya Mall,"Croma, Basement-Himalaya Mall, Drive In Rd, Ni...",11:00am-09:30pm,18005727662,23.053084,72.532931
1,Ghoddod Road,"Croma, Crossway Mall Ghod Dod Road, Ram Chowk,...",11:00am-09:00pm,18005727662,21.171958,72.790098
2,Croma Aundh,"Croma, Restaurant, Survey No.127/2B/1A Nanasah...",11:00am - 09:00pm,7208981676,18.562410,73.807188
3,Maxus Mall Bhayander,"Croma, Rashesh Mall, Near Maxus Mall, Sri Vall...",11:00am-09:00pm,7506716518,19.310268,72.853097
4,Bandra Linking Road,"Croma, Lower Ground Link Square Mall 490, Vitt...",11:00am-09:30pm,7045097408,19.065464,72.831740
...,...,...,...,...,...,...
187,Lucknow Aliganj,"Plot No.5, Sector E, Opposite Kendriya Bhawan ...",11:00am-09:00PM,8929991523,26.849842,80.967154
188,Rajahmundry Rtc Complex Road,"R.K. Plaza, Opp. Reliance Petrol Pump, Morampu...",11:00am-09:00pm,9606688013,17.006799,81.786881
189,Pune Katraj,"Showroom no 1, 2 & 3, Sugandhtara Realty, Surv...",10:00am-09:00pm,9607486969,18.453679,73.856320
190,Ahmedabad Vastral,"Croma, Satva Icon, Vastral cross road, Sardar ...",11:00am-09:30pm,7069009133,23.003544,72.647688


In [7]:
df.to_csv('Croma stores.csv', index = False)